In [1]:
#This is dev branch
import math
import copy
import os
import numpy as np
import pandas as pd

In [2]:
def readMultipleData(path,fileNamePrefix,headerNames,dataTypes):
    filesToExplore = []
    for file in os.listdir(path):
        if file.startswith(fileNamePrefix):
            filesToExplore.append(file)
            # print(f"{file} read")
        else:
            continue

    print(f"{len(filesToExplore)} files read")
    
    readData = []
    for files in filesToExplore:
        fileRead = pd.read_csv(path + files, sep='\t', names=headerNames,dtype=dataTypes)
        readData.append(fileRead)

    readData = pd.concat(readData, ignore_index=True)
    return readData

In [3]:
# now region Data
regionData = pd.read_csv('./training_data/cluster_map/cluster_map', sep='\t', names=['region_hash', 'region_id'],dtype={'region_hash': 'str', 'region_id': 'int'})
print(regionData.head())

                        region_hash  region_id
0  90c5a34f06ac86aee0fd70e2adce7d8a          1
1  f2c8c4bb99e6377d21de71275afd6cd2          2
2  58c7a4888306d8ff3a641d1c0feccbe3          3
3  b26a240205c852804ff8758628c0a86a          4
4  4b9e4cf2fbdc8281b8a1f9f12b80ce4d          5


In [4]:
# read order data
dataTypes = {'order_id':'str', 'driver_id':'str', 'passenger_id':'str', 'start_region_hash':'str', 'dest_region_hash':'str', 'price':'double', 'time':'str'}
orderDataPath = './training_data/order_data/'
orderData = readMultipleData(orderDataPath,'order', ['order_id', 'driver_id', 'passenger_id', 'start_region_hash', 'dest_region_hash', 'price', 'time'], dataTypes)
print("printing order data")
print(orderData.head())

# read weather data
dataTypes={'time':'str', 'weather':'int', 'temperature':'double', 'PM2.5':'double'}
weatherDataPath = './training_data/weather_data/'
weatherData = readMultipleData(weatherDataPath,'weather', ['time', 'weather', 'temperature', 'PM2.5'], dataTypes)
print("printing weather data")
print(weatherData.head())

order_data_2016-01-01 read
order_data_2016-01-02 read
order_data_2016-01-03 read
order_data_2016-01-04 read
order_data_2016-01-05 read
order_data_2016-01-06 read
order_data_2016-01-07 read
order_data_2016-01-08 read
order_data_2016-01-09 read
order_data_2016-01-10 read
order_data_2016-01-11 read
order_data_2016-01-12 read
order_data_2016-01-13 read
order_data_2016-01-14 read
order_data_2016-01-15 read
order_data_2016-01-16 read
order_data_2016-01-17 read
order_data_2016-01-18 read
order_data_2016-01-19 read
order_data_2016-01-20 read
order_data_2016-01-21 read
21 files read
printing order data
                           order_id                         driver_id  \
0  97ebd0c6680f7c0535dbfdead6e51b4b  dd65fa250fca2833a3a8c16d2cf0457c   
1  92c3ac9251cc9b5aab90b114a1e363be  c077e0297639edcb1df6189e8cda2c3d   
2  abeefc3e2aec952468e2fd42a1649640  86dbc1b68de435957c61b5a523854b69   
3  cb31d0be64cda3cc66b46617bf49a05c  4fadfa6eeaa694742de036dddf02b0c4   
4  139d492189ae5a933122c098f63252b

In [5]:
# read POI Data
# poiData = pd.read_csv('./training_data/poi_data/poi_data', sep='\t', names=['region_hash','poi_class'],dtype={'region_hash': 'str', 'poi_class': 'str'})
# print("printing poi data")
# print(poiData.head())


ParserError: Error tokenizing data. C error: Expected 139 fields in line 52, saw 142
